In [1]:
import copy
import random as rd
import csv

In [2]:
nodeDetails = {'node1':90,
               'node2':30,
               'node3':60,
               'node4':60,
               'node5':40,
               'node6':20,
               'node7':60,
               'node8':20,
               'end':999
              } #nodename : service tieme in seconds
nodeOccupancies = {}
nodeDwellRange = 10 #seconds eg. 40 second dwell time with 10 second range, people can stay between 30-50 seconds

liftRoundTime = 45 #seconds
liftCapacity = 12 #people

simulationTime = 1800 #seconds

nodeList = list(nodeDetails)[:-1]

In [3]:
class person:
    def __init__(self,personIndex,startNode,timeAtNode):
        self.idx = personIndex
        self.curNode = startNode
        self.curNodeIdx = 0
        self.timeAtCurNode = timeAtNode
        self.timeAtNextNode = timeAtNode + self.calculateTimeToNextNode()

    def __str__(self):
        return f"{self.idx} {self.curNode} {self.timeAtCurNode} {self.timeAtNextNode}"

    def calculateTimeToNextNode(self):
        lowRange = nodeDetails[self.curNode]-nodeDwellRange
        highRange = nodeDetails[self.curNode] + nodeDwellRange
        timeToSpendAtCurNode = rd.randrange(lowRange,highRange+1,1)
        return timeToSpendAtCurNode

    def updateAgentLocation(self,curTime):
        if curTime == self.timeAtNextNode:
            #Reached Last Step, remove details
            if self.curNode == nodeList[-1]:
                self.curNode = 'end'
                self.timeAtNextNode = -1
                return
            self.curNodeIdx+=1
            self.curNode = nodeList[self.curNodeIdx]
            self.timeAtCurNode = curTime
            self.timeAtNextNode = curTime + self.calculateTimeToNextNode()
            return

In [4]:
#simulate arrival time of people coming in a lift and moving through from one to the next and calculate occupancy of each node over time

#keep track of time when people arrive at a node, where they are and when they should leave.
numberOfPeople = 0
peopleList = []
for curTime in range(simulationTime):
    
    if curTime not in nodeOccupancies:
        nodeOccupancies[curTime] = {}
        for key in nodeDetails:
            nodeOccupancies[curTime][key] = 0

    if curTime%liftRoundTime == 0:# People Arrive at node 1
        for personIndex in range(liftCapacity):
            peopleList.append(person(len(peopleList),nodeList[0],curTime))

    for agent in peopleList:
        agent.updateAgentLocation(curTime)
        agentLocation = agent.curNode
        nodeOccupancies[curTime][agentLocation] += 1

In [5]:
# Get headers from one of the inner dictionaries
fieldnames = ['time'] + list(next(iter(nodeOccupancies.values())).keys())

with open('nodeStaticCalcsOutput.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for time, node_values in nodeOccupancies.items():
        row = {'time': time}
        row.update(node_values)
        writer.writerow(row)